In [22]:
import galsim
from matplotlib import pyplot as plt
plt.style.use(['dark_background'])
%matplotlib inline

import numpy as np

### HSM with simulated profiles

In [58]:
galaxy = galsim.Gaussian(flux=1.0, sigma=1.0).shear(g1=.01, g2=.03).drawImage(scale=0.2)

In [60]:
new_params = galsim.hsm.HSMParams(max_amoment=5.0e5)
moments = galsim.hsm.FindAdaptiveMom(img, hsmparams=new_params)

In [62]:
moments.observed_shape

galsim.Shear((0.009999914491696234+0.029999758391150814j))